In [17]:
!pip install python-dotenv

In [18]:
%%writefile .env
ANTHROPIC_API_KEY=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

Writing .env


In [24]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 4.4 MB/s eta 0:00:00


In [26]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.5 MB/s eta 0:00:00


In [19]:
import os
from dotenv import load_dotenv

In [20]:
# Load the .env file
load_dotenv(".env", override=True)

True

In [21]:
# Step 4: Access and print the API key
api_key = os.getenv("ANTHROPIC_API_KEY")
if api_key:
    print(f"API Key: {api_key}")
else:
    print("API Key not found in .env file.")

API Key: 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


In [22]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")

In [27]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState

In [28]:
class WeatherResponse(BaseModel):
    """ Respond to the user with this"""

    temperature: float = Field(description='The temperature in fahrenheit')
    wind_direction: str = Field(description='The direction of the wind in abbreviated form')
    wind_speed: float = Field(description='The Speed of the wind in km/h')

In [29]:
#Inherit 'messages' key from MessageState, which is a list of chat messages
class AgentState(MessagesState):
    # Final structured response from the agent
    final_response: WeatherResponse

@tool
def get_weather(city: Literal['nyc','sf']):
    """ Use this to get weather information"""

    if city == 'nyc':
        return 'It is cloudy in NYC, with 5 mph winds in the North-East direction and a temperature of 70 degrees'
    elif city == 'sf':
        return "It is 75 degress and sunny in SF, with 3 ph winds in the South-East direction"
    else:
        raise AssertionError("Unkown city")

tools = [get_weather]

model = ChatGroq(model='llama3-8b-8192')

model_with_tools = model.bind_tools(tools)
model_with_structured_output = model.with_structured_output(WeatherResponse)

Option 1 : Bind output as tool

Define graph


In [30]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

tools = [get_weather, WeatherResponse]

#Force the model to use tools by passing tool_choice = 'any'

model_with_response_tool = model.bind_tools(tools,
                                            tool_choice='any')

# Define the function that calls the model
def call_model(state: AgentState):
    response = model_with_response_tool.invoke(state['messages'])

    # We return a list, because this will get added to the existing list
    return {'messages': [response]}

# Define the function that reponds to user
def respond(state: AgentState):
    # Construct the final answer from the arguments of the last tool call
    weather_tool_call = state['messages'][-1].tool_calls[0]
    response = WeatherResponse(**weather_tool_call['args'])

    #since we are using tool calling to return structured output
    # we need to add a tool message corresponding to the WeatherResponse tool call,
    # This is due to LLM providers' requirement that AI messages with tool calls
    # need to be follwed by a tool message for each tool call

    tool_message = {
        'type' : 'tool',
        'content': 'Here is your structured response',
        'tool_call_id' : weather_tool_call['id'],
    }

    # We return the final answer
    return {'final_response': response,
            'messages' : [tool_message]}

def should_continue(state: AgentState):
    message = state['messages']
    last_message = message[-1]
    # If there is only tool call and it is the response tool call we respond to the user

    if (
        len(last_message.tool_calls) == 1
        and
        last_message.tool_calls[0]['name'] == 'WeatherResponse'
    ):
        return 'respond'
    # Otherwise we will use the tool node again
    else:
        return 'continue'

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("respond",respond)
workflow.add_node('tools', ToolNode(tools))

# Set the entrypoint as 'agent'
# This means that this node is the first one called
workflow.set_entry_point('agent')

# We now add a conditional edge
workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue': 'tools',
        'respond' : 'respond'
    }
)

workflow.add_edge('tools','agent')
workflow.add_edge('respond',END)
graph = workflow.compile()

Usage


In [31]:
answer = graph.invoke(input={
                            'messages': [('human',
                                        "what's the weather in SF")]
                            })['final_response']

In [32]:
answer

WeatherResponse(temperature=75.0, wind_direction='SE', wind_speed=3.0)

Option 2:2 LLMS



Define Graph

second LLM to force structured output.

In [33]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage


# Define the function that calls the model
def call_model(state: AgentState):
    response = model_with_tools.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the function that responds to the user
def respond(state: AgentState):
    # We call the model with structured output in order to return the same format to the user every time
    # state['messages'][-2] is the last ToolMessage in the convo, which we convert to a HumanMessage for the model to use
    # We could also pass the entire chat history, but this saves tokens since all we care to structure is the output of the tool
    response = model_with_structured_output.invoke(
        [HumanMessage(content=state["messages"][-2].content)]
    )
    # We return the final answer
    return {"final_response": response}


# Define the function that determines whether to continue or not
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we respond to the user
    if not last_message.tool_calls:
        return "respond"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("respond", respond)
workflow.add_node("tools", ToolNode(tools))

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "respond": "respond",
    },
)

workflow.add_edge("tools", "agent")
workflow.add_edge("respond", END)
graph = workflow.compile()

Usage

In [34]:
answer = graph.invoke(input={"messages": [("human", "what's the weather in SF?")]})[
    "final_response"
]

In [35]:
answer

WeatherResponse(temperature=75.0, wind_direction='SE', wind_speed=3.0)